In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
import json
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# TensorFlow settings
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Project root path
project_root = Path('../')
sys.path.append(str(project_root))

# Import custom modules
from src.lstm_v2_model import LSTMv2HybridModel, create_lstm_v2_hybrid_model
from src.lstm_v2_trainer import LSTMv2Trainer

print("Libraries imported successfully")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")


2025-07-06 23:19:16.122861: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-06 23:19:17.878863: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751811558.441148  244602 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751811558.593063  244602 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751811559.855480  244602 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Libraries imported successfully
TensorFlow version: 2.19.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Experiment configuration
EXPERIMENT_NAME = "lstm_v2_hybrid"
WINDOW_CONFIG = "w64_s16"  # or "w128_s32"
RANDOM_STATE = 42

# Set random seeds for reproducibility
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Window Config: {WINDOW_CONFIG}")
print(f"Random State: {RANDOM_STATE}")


Experiment: lstm_v2_hybrid
Window Config: w64_s16
Random State: 42


In [3]:
# Initialize trainer
trainer = LSTMv2Trainer(EXPERIMENT_NAME, WINDOW_CONFIG)

# Load preprocessed data
data = trainer.load_preprocessed_data()

print("\n=== Data Summary ===")
print(f"Sensor windows shape: {data['X_sensor_windows'].shape}")
print(f"Demographics shape: {data['X_demographics_windows'].shape}")
print(f"Labels shape: {data['y_windows'].shape}")
print(f"Number of classes: {len(data['label_encoder'].classes_)}")
print(f"Classes: {data['label_encoder'].classes_}")


LSTM v2学習環境初期化完了
実験名: lstm_v2_hybrid
ウィンドウ設定: w64_s16
出力ディレクトリ: ../output/experiments/lstm_v2_hybrid_w64_s16
前処理済みデータ: ../output/experiments/lstm_v2_w64_s16/preprocessed
前処理済みデータを読み込み中...
データ読み込み完了:
  センサーデータ: (13393, 64, 332)
  Demographics: (13393, 20)
  ラベル: (13393,)
  クラス数: 18

=== Data Summary ===
Sensor windows shape: (13393, 64, 332)
Demographics shape: (13393, 20)
Labels shape: (13393,)
Number of classes: 18
Classes: ['Above ear - pull hair' 'Cheek - pinch skin' 'Drink from bottle/cup'
 'Eyebrow - pull hair' 'Eyelash - pull hair'
 'Feel around in tray and pull out an object' 'Forehead - pull hairline'
 'Forehead - scratch' 'Glasses on/off' 'Neck - pinch skin'
 'Neck - scratch' 'Pinch knee/leg skin' 'Pull air toward your face'
 'Scratch knee/leg skin' 'Text on phone' 'Wave hello' 'Write name in air'
 'Write name on leg']


In [4]:
# Model hyperparameters
MODEL_PARAMS = {
    'lstm_units_1': 64,
    'lstm_units_2': 32,
    'dense_units': 32,
    'demographics_dense_units': 16,
    'fusion_dense_units': 24,
    'dropout_rate': 0.3,
    'dense_dropout_rate': 0.2,
    'learning_rate': 0.001,
    'batch_size': 32,
    'epochs': 50,
    'patience': 15,
    'reduce_lr_patience': 8,
    'use_tqdm': True,
    'use_tensorboard': True
}

print("Model Parameters:")
for key, value in MODEL_PARAMS.items():
    print(f"  {key}: {value}")


Model Parameters:
  lstm_units_1: 64
  lstm_units_2: 32
  dense_units: 32
  demographics_dense_units: 16
  fusion_dense_units: 24
  dropout_rate: 0.3
  dense_dropout_rate: 0.2
  learning_rate: 0.001
  batch_size: 32
  epochs: 50
  patience: 15
  reduce_lr_patience: 8
  use_tqdm: True
  use_tensorboard: True


In [5]:
# Compare different fusion methods
print("Comparing different fusion methods...")
print("This will train 3 models: concatenate, attention, and gated fusion")
print("Each model will be trained for up to 50 epochs with early stopping")

comparison_results = trainer.compare_fusion_methods(data, MODEL_PARAMS)

print("\n=== Fusion Methods Comparison ===")
for fusion_type, results in comparison_results.items():
    if results is not None:
        print(f"\n{fusion_type.upper()} Fusion:")
        print(f"  Accuracy: {results['test_accuracy']:.4f}")
        print(f"  F1-macro: {results['f1_macro']:.4f}")
        print(f"  F1-weighted: {results['f1_weighted']:.4f}")
        print(f"  Loss: {results['test_loss']:.4f}")
    else:
        print(f"\n{fusion_type.upper()} Fusion: ERROR")


Comparing different fusion methods...
This will train 3 models: concatenate, attention, and gated fusion
Each model will be trained for up to 50 epochs with early stopping
異なる融合方式の比較実験開始...

=== CONCATENATE 融合方式 ===
ハイブリッドモデル学習開始 (融合方式: concatenate)...
データ分割中...
テストサイズ: 0.2
検証サイズ: 0.2
データ分割完了:
  訓練 - センサー: (8571, 64, 332), Demographics: (8571, 20), ラベル: (8571,)
  検証 - センサー: (2143, 64, 332), Demographics: (2143, 20), ラベル: (2143,)
  テスト - センサー: (2679, 64, 332), Demographics: (2679, 20), ラベル: (2679,)
入力形状:
  センサー: (64, 332)
  Demographics: (20,)
  クラス数: 18
GPU利用可能: 1台
LSTM v2ハイブリッドモデル初期化完了
センサー入力形状: (64, 332)
Demographics入力形状: (20,)
クラス数: 18
融合方式: concatenate
GPU利用可能: True
LSTM v2ハイブリッドモデルを構築中...


I0000 00:00:1751811613.454332  244602 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5660 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:01:00.0, compute capability: 8.6


モデル構築完了
総パラメータ数: 117,722

=== モデルサマリー ===


Model: "lstm_v2_hybrid"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sensor_input        │ (None, 64, 332)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64, 64)    │    101,632 │ sensor_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_bn_1           │ (None, 64, 64)    │        256 │ lstm_1[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_dropout_1      │ (None, 64, 64)    │          0 │ lstm_bn_1[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 32)        │     12,416 │ lstm_dropout_1[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_input  │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_bn_2           │ (None, 32)        │        128 │ lstm_2[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dense… │ (None, 16)        │        336 │ demographics_inp… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_dropout_2      │ (None, 32)        │          0 │ lstm_bn_2[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dropo… │ (None, 16)        │          0 │ demographics_den… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sensor_dense        │ (None, 32)        │      1,056 │ lstm_dropout_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dense… │ (None, 16)        │        272 │ demographics_dro… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sensor_dropout      │ (None, 32)        │          0 │ sensor_dense[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dropo… │ (None, 16)        │          0 │ demographics_den… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feature_fusion      │ (None, 48)        │          0 │ sensor_dropout[0… │
│ (Concatenate)       │                   │            │ demographics_dro… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dense        │ (None, 24)        │      1,176 │ feature_fusion[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dropout      │ (None, 24)        │          0 │ fusion_dense[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

...


Epoch 16: val_loss did not improve from 1.57090

Epoch 17: val_loss improved from 1.57090 to 1.54771, saving model to ../output/experiments/lstm_v2_hybrid_w64_s16/models/lstm_v2_hybrid_gated_best.h5



Epoch 18: val_loss did not improve from 1.54771

Epoch 19: val_loss improved from 1.54771 to 1.50481, saving model to ../output/experiments/lstm_v2_hybrid_w64_s16/models/lstm_v2_hybrid_gated_best.h5



Epoch 20: val_loss did not improve from 1.50481

Epoch 21: val_loss did not improve from 1.50481

Epoch 22: val_loss did not improve from 1.50481

Epoch 23: val_loss did not improve from 1.50481

Epoch 24: val_loss did not improve from 1.50481

Epoch 25: val_loss did not improve from 1.50481

Epoch 26: val_loss did not improve from 1.50481

Epoch 27: val_loss did not improve from 1.50481

Epoch 27: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 28: val_loss improved from 1.50481 to 1.48911, saving model to ../output/experiments/lstm_v2_hybrid_w64_s16/models/lstm_v2_hybrid_gated_best.h5



Epoch 29: val_loss did not improve from 1.48911

Epoch 30: val_loss improved from 1.48911 to 1.47744, saving model to ../output/experiments/lstm_v2_hybrid_w64_s16/models/lstm_v2_hybrid_gated_best.h5



Epoch 31: val_loss did not improve from 1.47744

Epoch 32: val_loss did not improve from 1.47744

Epoch 33: val_loss did not improve from 1.47744

Epoch 34: val_loss did not improve from 1.47744

Epoch 35: val_loss did not improve from 1.47744

Epoch 36: val_loss did not improve from 1.47744

Epoch 37: val_loss did not improve from 1.47744

Epoch 38: val_loss did not improve from 1.47744

Epoch 38: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 39: val_loss did not improve from 1.47744

Epoch 40: val_loss did not improve from 1.47744

Epoch 41: val_loss did not improve from 1.47744

Epoch 42: val_loss did not improve from 1.47744

Epoch 43: val_loss did not improve from 1.47744

Epoch 44: val_loss did not improve from 1.47744

Epoch 45: val_loss did not improve from 1.47744
Epoch 45: early stopping
Restoring model weights from the end of the best epoch: 30.
学習完了！学習時間: 371.48秒
最良検証Loss: 1.4774
最良検証Accuracy: 0.5539
ハイブリッドモデル評価中...
テストLoss: 1.5273
テストAccuracy: 

In [6]:
# Find best performing model
if comparison_results:
    valid_results = {k: v for k, v in comparison_results.items() if v is not None}
    
    if valid_results:
        # Best model by F1-macro score
        best_fusion_type = max(valid_results.keys(), key=lambda x: valid_results[x]['f1_macro'])
        best_results = valid_results[best_fusion_type]
        
        print(f"\n=== Best Model: {best_fusion_type.upper()} Fusion ===")
        print(f"Accuracy: {best_results['test_accuracy']:.4f}")
        print(f"F1-Macro: {best_results['f1_macro']:.4f}")
        print(f"F1-Weighted: {best_results['f1_weighted']:.4f}")
        print(f"Loss: {best_results['test_loss']:.4f}")
        
        # Save best model info
        best_model_info = {
            'best_fusion_type': best_fusion_type,
            'best_results': best_results,
            'model_params': MODEL_PARAMS,
            'experiment_name': EXPERIMENT_NAME,
            'window_config': WINDOW_CONFIG,
            'timestamp': datetime.now().isoformat()
        }
        
        best_model_path = trainer.results_dir / 'best_model_info.json'
        with open(best_model_path, 'w') as f:
            json.dump(best_model_info, f, indent=2)
        
        print(f"\nBest model info saved to: {best_model_path}")
    else:
        print("No valid results to compare")
else:
    print("No comparison results available")



=== Best Model: GATED Fusion ===
Accuracy: 0.5293
F1-Macro: 0.4488
F1-Weighted: 0.5006
Loss: 1.5273

Best model info saved to: ../output/experiments/lstm_v2_hybrid_w64_s16/results/best_model_info.json


In [7]:
print("\n" + "="*50)
print("LSTM v2 HYBRID MODEL EXPERIMENT SUMMARY")
print("="*50)

print(f"\nExperiment Configuration:")
print(f"  Name: {EXPERIMENT_NAME}")
print(f"  Window Config: {WINDOW_CONFIG}")
print(f"  Total Samples: {len(data['y_windows'])}")
print(f"  Sensor Features: {data['X_sensor_windows'].shape[2]}")
print(f"  Demographics Features: {data['X_demographics_windows'].shape[1]}")
print(f"  Total Classes: {len(data['label_encoder'].classes_)}")

print(f"\nModel Architecture:")
print(f"  LSTM Layers: 2 ({MODEL_PARAMS['lstm_units_1']}, {MODEL_PARAMS['lstm_units_2']} units)")
print(f"  Dense Layers: Multiple with regularization")
print(f"  Fusion Methods: Concatenate, Attention, Gated")
print(f"  Dropout: {MODEL_PARAMS['dropout_rate']}")
print(f"  Learning Rate: {MODEL_PARAMS['learning_rate']}")

if 'valid_results' in locals() and valid_results:
    print(f"\nBest Performance:")
    print(f"  Best Method: {best_fusion_type.title()} Fusion")
    print(f"  Test Accuracy: {best_results['test_accuracy']:.4f}")
    print(f"  F1-Macro: {best_results['f1_macro']:.4f}")
    print(f"  F1-Weighted: {best_results['f1_weighted']:.4f}")

print(f"\nOutput Directory: {trainer.output_dir}")
print(f"Models Directory: {trainer.models_dir}")
print(f"Results Directory: {trainer.results_dir}")

print("\nNext Steps:")
print("1. Create inference script for Kaggle submission")
print("2. Consider hyperparameter tuning")
print("3. Experiment with different architectures")
print("4. Compare with LSTM v1 baseline")

print("\nExperiment completed successfully!")



LSTM v2 HYBRID MODEL EXPERIMENT SUMMARY

Experiment Configuration:
  Name: lstm_v2_hybrid
  Window Config: w64_s16
  Total Samples: 13393
  Sensor Features: 332
  Demographics Features: 20
  Total Classes: 18

Model Architecture:
  LSTM Layers: 2 (64, 32 units)
  Dense Layers: Multiple with regularization
  Fusion Methods: Concatenate, Attention, Gated
  Dropout: 0.3
  Learning Rate: 0.001

Best Performance:
  Best Method: Gated Fusion
  Test Accuracy: 0.5293
  F1-Macro: 0.4488
  F1-Weighted: 0.5006

Output Directory: ../output/experiments/lstm_v2_hybrid_w64_s16
Models Directory: ../output/experiments/lstm_v2_hybrid_w64_s16/models
Results Directory: ../output/experiments/lstm_v2_hybrid_w64_s16/results

Next Steps:
1. Create inference script for Kaggle submission
2. Consider hyperparameter tuning
3. Experiment with different architectures
4. Compare with LSTM v1 baseline

Experiment completed successfully!
